In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import csv

from sklearn.cluster import MiniBatchKMeans
# from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import glob
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import metrics
from commonFunctions import *

## Extract keypoints from each image

In [ ]:
img_path = '../input/images/'
train = pd.read_csv('../input/train.csv')
species = train.species.sort_values().unique()

dico = []

def step1():
    for leaf in train.id:
        img = cv2.imread(img_path + str(leaf) + ".jpg")
        kp, des = cv2.sift.detectAndCompute(img, None)

        for d in des:
            dico.append(d)

In [2]:
def shadow_removalDialate(img):
    # img2 = cv2.imread(f'../Dataset_0-5/men/{num}/4_men (49).JPG')
    img2 = cv2.resize(img, (256, 256))
    
    img_hls = cv2.cvtColor(img2, cv2.COLOR_BGR2HLS)

    # define lower and upper bounds for blue color in HLS format
    lower_blue = np.array([0, 0, 60])
    upper_blue = np.array([20, 255, 255])

    # create a mask for blue color in HLS format
    mask = cv2.inRange(img_hls, lower_blue, upper_blue)
    # daialation
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.dilate(mask,kernel,iterations = 8)
    # apply the mask to the original image
    result = cv2.bitwise_and(img2, img2, mask=mask)
    # show_images([img2, mask, result], ["Original Image", "dialation", "masked"])
    # io.imshow(img2)
    return result

In [3]:
def preprocessing(img,img2):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization to enhance contrast and remove lighting effects
    equalized = cv2.equalizeHist(gray)
    # io.imshow(equalized)
    # # Find the contours of the binary image
    contours, hierarchy = cv2.findContours(equalized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=cv2.contourArea)

    # # Draw the largest contour on the original image
    cv2.drawContours(img2, [max_contour], 0, (0, 255, 0), 2)
    # io.imshow(img2)
    # # Show the image with the largest contour drawn
    # # Create a bounding box around the hand
    x, y, w, h = cv2.boundingRect(max_contour)

    # # Crop the image to the bounding box around the hand
    hand = equalized[y:y+h, x:x+w]
    img3 = cv2.resize(hand, (256, 256))
    # io.imshow(img3)

    return img3

In [4]:
arr = ["0","1","2","3","4","5"]
dico = []
lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))
            result_image=shadow_removalDialate(img)
            preprocessed_image=preprocessing(result_image,img)
            
            
            sift = cv2.ORB_create()
            kp, des = sift.detectAndCompute(preprocessed_image, None)
            for d in des:
                dico.append(d)
        except:
            pass
        

In [5]:
arr = ["0","1","2","3","4","5"]
lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/Women/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))
            result_image=shadow_removalDialate(img)
            preprocessed_image=preprocessing(result_image,img)
            
            
            sift = cv2.ORB_create()
            kp, des = sift.detectAndCompute(preprocessed_image, None)
            for d in des:
                dico.append(d)
        except:
            pass
        

In [1]:
import numpy as np
import cv2

# Load image and detect SIFT keypoints and descriptors
gray = cv2.imread('test.png', 0)
sift = cv2.ORB_create()
kp, des = sift.detectAndCompute(gray, None)

# Stack descriptors into a NumPy array
# des_array = np.vstack(des)
dd = []
for d in des:
    dd.append(d)
# print(np.shape(des_array))
print(np.shape(dd))


(500, 32)


In [35]:
img = cv2.imread('test.png')
gray= cv2.cvtColor(img,cv.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
kp = sift.detect(gray,None)
des = sift.compute(gray,kp)
des = np.array(des)

# desc_samples = des[np. random. randint(len(des), size = 20)]
# print(desc_samples)
print(np.shape(des))
img=cv2.drawKeypoints(gray,kp,img)
# cv2.imwrite('sift_keypoints.jpg',img)
# show_images([img])

C:\Users\khale\AppData\Local\Temp\ipykernel_3948\479942303.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  des = np.array(des)


ValueError: could not broadcast input array from shape (608,128) into shape (608,)

## Clustering
We now have an array with a huge number of descriptors. We cannot use all of them to create or model so we need to cluster them. A rule-of-thumb is to create k centers with k = number of categories * 10 (in our case, it's 60).

In [ ]:
def step2():
    k = np.size(species) * 10

    batch_size = np.size(os.listdir(img_path)) * 3
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
    
# I use MiniBatchKMeans to avoid Memory Error.

In [15]:
print(np.shape(dico))

(693307, 32)


In [1]:
# print(np.shape(dico[0]))
# print(np.shape(dico[1]))
# print(np.shape(dico[2]))
# print(np.shape(dico[3]))
# print(np.shape(dico[4]))
# print(np.shape(dico[5]))
# print(np.shape(dico[6]))
# print(np.shape(dico[7]))
# print(np.shape(dico[8]))
# print(np.shape(dico[9]))

# dico2 = dico[1::2]
# for i in range(len(dico2)):
#     print(np.shape(dico2[i]))


In [17]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=60)
kmeans.fit(dico)
joblib.dump(kmeans, "model/Tkmeans.pkl")

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


['model/Tkmeans.pkl']

In [4]:
k = 10 * 6 
batch_size = 50
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
# kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1, max_iter=500).fit(dico)

joblib.dump(kmeans, "model/Bkmeans.pkl")

c:\Users\khale\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Init 1/3 with method k-means++
Inertia for init 1/3: 24316744.0
Init 2/3 with method k-means++
Inertia for init 2/3: 24446508.0
Init 3/3 with method k-means++
Inertia for init 3/3: 23965083.0
[MiniBatchKMeans] Reassigning 25 cluster centers.
Minibatch step 1/729348: mean batch inertia: 158672.54
[MiniBatchKMeans] Reassigning 24 cluster centers.
Minibatch step 2/729348: mean batch inertia: 161024.95, ewa inertia: 161024.95
Minibatch step 3/729348: mean batch inertia: 150086.0149728395, ewa inertia: 161021.95036058762
Minibatch step 4/729348: mean batch inertia: 137389.70503047053, ewa inertia: 161015.47000264423
Minibatch step 5/729348: mean batch inertia: 135406.26182187418, ewa inertia: 161008.44752833678
Minibatch step 6/729348: mean batch inertia: 124876.87317147077, ewa inertia: 160998.53964478103
Minibatch step 7/729348: mean batch inertia: 131604.19429286575, ewa inertia: 160990.4792223907
Minibatch step 8/729348: mean batch inertia: 128878.91889972937, ewa inertia: 160981.673694

['model/Bkmeans.pkl']

## Creation of the histograms
To create our each image by a histogram. We will create a vector of k value for each image. For each keypoints in an image, we will find the nearest center and increase by one its value.

In [ ]:
def step3():
    kmeans.verbose = False

    histo_list = []

    for leaf in train.id:
        img = cv2.imread(img_path + str(leaf) + ".jpg")
        kp, des = sift.detectAndCompute(img, None)

        histo = np.zeros(k)
        nkp = np.size(kp)

        for d in des:
            idx = kmeans.predict([d])
            histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

        histo_list.append(histo)

In [18]:
# kmeans.verbose = False
# kmeans=joblib.load("model/kmeans.pkl")
histo_list = []
Y = []
arr = ["0","1","2","3","4","5"]

lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/men/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))
            result_image=shadow_removalDialate(img)
            preprocessed_image=preprocessing(result_image,img)
            
            
            sift = cv2.ORB_create()
            kp, des = sift.detectAndCompute(preprocessed_image, None)
            histo = np.zeros(60)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

            histo_list.append(histo)
            Y.append(int(label))
        except:
            pass
        

In [19]:
# kmeans.verbose = False
# kmeans=joblib.load("model/kmeans.pkl")
arr = ["0","1","2","3","4","5"]

lower_blue = np.array([0, 0, 65])
upper_blue = np.array([20, 255, 255])


for label in arr:
    dirList = glob.glob("../Dataset_0-5/Women/"+str(label)+"/*.jpg")
    
    for img_path in dirList:
        try:
            im= cv2.imread(img_path)
            img = cv2.resize(im, (256,256))
            result_image=shadow_removalDialate(img)
            preprocessed_image=preprocessing(result_image,img)
            
            
            sift = cv2.ORB_create()
            kp, des = sift.detectAndCompute(preprocessed_image, None)
            histo = np.zeros(60)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

            histo_list.append(histo)
            Y.append(int(label))
        except:
            pass
        

## train our model

In [12]:
print(Y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [20]:
print(np.shape(histo_list))
Y = [int(num) for num in Y]

(1821, 60)


In [21]:
X = np.array(histo_list)
train_x,test_x,train_y,test_y = train_test_split(histo_list,Y, test_size = 0.2)
classifier=SVC(kernel="poly", random_state=6, degree=6)
classifier.fit(train_x,train_y)
joblib.dump(classifier, "model/Bsift_model.pkl")

['model/Bsift_model.pkl']

In [22]:
from sklearn import metrics

prediction=classifier.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.4273972602739726
Precision=  0.4263580334744602
Recall=  0.4273972602739726
F1 Score=  0.4253695306063265
Confusion Matrix=  [[44  7  4  3  2  0]
 [12 25 15  8  4  4]
 [ 4 15 19 17  4  2]
 [ 5  6  9 16 16  6]
 [ 3  3 10 17 17  9]
 [ 2  1  5  4 12 35]]
Classification Report=                precision    recall  f1-score   support

           0       0.63      0.73      0.68        60
           1       0.44      0.37      0.40        68
           2       0.31      0.31      0.31        61
           3       0.25      0.28      0.26        58
           4       0.31      0.29      0.30        59
           5       0.62      0.59      0.61        59

    accuracy                           0.43       365
   macro avg       0.43      0.43      0.43       365
weighted avg       0.43      0.43      0.43       365



In [23]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier and fit it to the training data
rfc = RandomForestClassifier(n_estimators=300)
rfc.fit(train_x, train_y)


RandomForestClassifier(n_estimators=300)

In [24]:
prediction=rfc.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.4712328767123288
Precision=  0.5042776366054734
Recall=  0.4712328767123288
F1 Score=  0.48357732907686685
Confusion Matrix=  [[46  8  6 10  0  1]
 [ 9 32 15  7  4  3]
 [ 3  9 17 12  6  4]
 [10  2 11 16 12  1]
 [ 1  4  7 15 18  4]
 [ 1  2  6  5 15 43]]
Classification Report=                precision    recall  f1-score   support

           0       0.66      0.65      0.65        71
           1       0.56      0.46      0.50        70
           2       0.27      0.33      0.30        51
           3       0.25      0.31      0.27        52
           4       0.33      0.37      0.35        49
           5       0.77      0.60      0.67        72

    accuracy                           0.47       365
   macro avg       0.47      0.45      0.46       365
weighted avg       0.50      0.47      0.48       365



In [25]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=150)
gbc.fit(train_x, train_y)

GradientBoostingClassifier(n_estimators=150)

In [26]:
prediction=gbc.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.40821917808219177
Precision=  0.41626814956175145
Recall=  0.40821917808219177
F1 Score=  0.41087252614329356
Confusion Matrix=  [[37  8  9  5  3  0]
 [11 23 15  5  2  3]
 [ 9 13 18 14  5  4]
 [ 9  4 13 17 12  6]
 [ 3  7  5 19 15  4]
 [ 1  2  2  5 18 39]]
Classification Report=                precision    recall  f1-score   support

           0       0.53      0.60      0.56        62
           1       0.40      0.39      0.40        59
           2       0.29      0.29      0.29        63
           3       0.26      0.28      0.27        61
           4       0.27      0.28      0.28        53
           5       0.70      0.58      0.63        67

    accuracy                           0.41       365
   macro avg       0.41      0.40      0.40       365
weighted avg       0.42      0.41      0.41       365



In [31]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=12)
knn.fit(train_x, train_y)

KNeighborsClassifier(n_neighbors=12)

In [32]:
prediction=knn.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.4010989010989011
Precision=  0.41718737863425037
Recall=  0.4010989010989011
F1 Score=  0.40549756965723355
Confusion Matrix=  [[29 10 10  7  2  3]
 [ 3 22 14  2  3  1]
 [ 7  9 18 10 10  3]
 [ 6 12 14 27 21  4]
 [ 4  3  9 10 16 14]
 [ 0  1  4 10 12 34]]
Classification Report=                precision    recall  f1-score   support

           0       0.59      0.48      0.53        61
           1       0.39      0.49      0.43        45
           2       0.26      0.32      0.29        57
           3       0.41      0.32      0.36        84
           4       0.25      0.29      0.27        56
           5       0.58      0.56      0.57        61

    accuracy                           0.40       364
   macro avg       0.41      0.41      0.41       364
weighted avg       0.42      0.40      0.41       364



In [33]:
from sklearn.ensemble import AdaBoostClassifier

adab_clf = AdaBoostClassifier(random_state=42)
adab_clf.fit(train_x, train_y)

AdaBoostClassifier(random_state=42)

In [34]:
prediction=adab_clf.predict(test_x)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.34615384615384615
Precision=  0.3664629109653014
Recall=  0.34615384615384615
F1 Score=  0.352662302327955
Confusion Matrix=  [[25 17 10  8  2  1]
 [ 8 23 24  8  6  3]
 [ 3 11 14  8 12  2]
 [10  4 11 22 14  9]
 [ 3  1  5  9 15 17]
 [ 0  1  5 11 15 27]]
Classification Report=                precision    recall  f1-score   support

           0       0.51      0.40      0.45        63
           1       0.40      0.32      0.36        72
           2       0.20      0.28      0.24        50
           3       0.33      0.31      0.32        70
           4       0.23      0.30      0.26        50
           5       0.46      0.46      0.46        59

    accuracy                           0.35       364
   macro avg       0.36      0.34      0.35       364
weighted avg       0.37      0.35      0.35       364



In [35]:
import xgboost as xgb
# Train an XGBoost Classifier on the training set
dtrain = xgb.DMatrix(train_x, label=train_y)
dtest = xgb.DMatrix(test_x, label=test_y)

param = {'max_depth': 3, 'eta': 0.1, 'objective': 'multi:softmax', 'num_class': 6}
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [36]:
prediction=bst.predict(dtest)
print("Accuracy= ",metrics.accuracy_score(prediction, test_y))
print("Precision= ",metrics.precision_score(prediction, test_y, average='weighted'))
print("Recall= ",metrics.recall_score(prediction, test_y, average='weighted'))
print("F1 Score= ",metrics.f1_score(prediction, test_y, average='weighted'))
print("Confusion Matrix= ",metrics.confusion_matrix(prediction, test_y))
print("Classification Report= ",metrics.classification_report(prediction, test_y))

Accuracy=  0.38461538461538464
Precision=  0.48895322760906257
Recall=  0.38461538461538464
F1 Score=  0.4130230089400787
Confusion Matrix=  [[35 17 13 13  8  4]
 [ 4 26 30 15  5  2]
 [ 2  3 12 10  5  5]
 [ 2  4  4  9  7  3]
 [ 5  2  4  8 20  7]
 [ 1  5  6 11 19 38]]
Classification Report=                precision    recall  f1-score   support

         0.0       0.71      0.39      0.50        90
         1.0       0.46      0.32      0.37        82
         2.0       0.17      0.32      0.23        37
         3.0       0.14      0.31      0.19        29
         4.0       0.31      0.43      0.36        46
         5.0       0.64      0.47      0.55        80

    accuracy                           0.38       364
   macro avg       0.41      0.38      0.37       364
weighted avg       0.49      0.38      0.41       364



In [ ]:
def step4():
    X = np.array(histo_list)
    Y = []

    # It's a way to convert species name into an integer
    for s in train.species:
        Y.append(np.min(np.nonzero(species == s)))

    mlp = MLPClassifier(verbose=True, max_iter=600000)
    mlp.fit(X, Y)

I also run this script with ORB instead of SIFT and I got best results. To do it, just replace `cv2.xfeatures2d.SIFT_create()` by `cv2.ORB_create().`